<a href="https://colab.research.google.com/github/ANUNAYBAGGA/Stock-Reserch/blob/main/StocksAI_phase1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [159]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [160]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [162]:
import os
import glob

path = 'gdrive/My Drive/stonks_DB'
extension = 'csv'
#os.chdir(path) #Use for first time only
result = glob.glob('*.{}'.format(extension))
print(result)
db = {}
for i in result:
  db[i[:-4]] = pd.read_csv(i)

print(db.keys())
stocks = list(db.keys())

['ONGC.csv', 'GAIL.csv', 'MARUTI.csv', 'HEROMOTOCO.csv', 'NESTLEIND.csv', 'ULTRACEMCO.csv', 'HINDUNILVR.csv', 'IOC.csv', 'LT.csv', 'RELIANCE.csv', 'BPCL.csv', 'ADANIPORTS.csv', 'BAJFINANCE.csv', 'M&M.csv', 'INFY.csv', 'TCS.csv', 'NTPC.csv', 'HDFC.csv', 'JSWSTEEL.csv', 'HCLTECH.csv', 'POWERGRID.csv', 'SBIN.csv', 'SHREECEM.csv', 'TATASTEEL.csv', 'HINDALCO.csv', 'UPL.csv']
dict_keys(['ONGC', 'GAIL', 'MARUTI', 'HEROMOTOCO', 'NESTLEIND', 'ULTRACEMCO', 'HINDUNILVR', 'IOC', 'LT', 'RELIANCE', 'BPCL', 'ADANIPORTS', 'BAJFINANCE', 'M&M', 'INFY', 'TCS', 'NTPC', 'HDFC', 'JSWSTEEL', 'HCLTECH', 'POWERGRID', 'SBIN', 'SHREECEM', 'TATASTEEL', 'HINDALCO', 'UPL'])


In [163]:
#Weekly
def shift_returns(shift):
  days_to_shift = shift
  print(db.keys())
  for i in db.keys():
    shifted_db = db[i].shift(days_to_shift)
    returns = shifted_db['adjusted_close'] - db[i]['adjusted_close']
    log_returns = np.log(shifted_db['adjusted_close'] / db[i]['adjusted_close'])
    db[i]['return'] = returns
    db[i]['log_return'] = log_returns


Momentum Based Trading System.
Choose the top and bottom performers and then long and short them respectively.

In [168]:
#Momentum Trading
days = 690
shift_returns(5)
print(db['ONGC'].head(10))
table = []
for i in range(days):
  signal = [0 for i in range(len(stocks))]
  order = [db[x]['log_return'][i] for x in stocks]
  order = pd.DataFrame(order).nlargest(3,0).index
  for x in order:
    signal[x] = 1
  table.append(signal)
table = pd.DataFrame(table , columns = stocks)
#print(table)

count = [table[stock].value_counts()[1] for stock in stocks]
count = pd.DataFrame(count)
long_momentum = [stocks[x] for x in count.nlargest(3,0).index]
short_momentum = [stocks[x] for x in count.nsmallest(3,0).index]
print("Score for each stock : ")
#print(count)
map = {}
for i in range(len(stocks)):
  map[count[0][i]] = map.get(count[0][i] , []) + [stocks[i]]
tmp = list(map.keys())
tmp.sort()
tmp.reverse()
map_movement = {}
for i in tmp:
  print(map[i] , " : ", i/days)
  map_movement[map[i][0]] = i/days

print("Rising stocks, Top stocks to long are  : " , long_momentum)
print("Dropping stocks, Top stocks to short are : ", short_momentum)
print(map_movement)

dict_keys(['ONGC', 'GAIL', 'MARUTI', 'HEROMOTOCO', 'NESTLEIND', 'ULTRACEMCO', 'HINDUNILVR', 'IOC', 'LT', 'RELIANCE', 'BPCL', 'ADANIPORTS', 'BAJFINANCE', 'M&M', 'INFY', 'TCS', 'NTPC', 'HDFC', 'JSWSTEEL', 'HCLTECH', 'POWERGRID', 'SBIN', 'SHREECEM', 'TATASTEEL', 'HINDALCO', 'UPL'])
    timestamp    open    high  ...  split_coefficient  return  log_return
0  2021-03-05  116.25  118.20  ...                1.0     NaN         NaN
1  2021-03-04  113.80  117.05  ...                1.0     NaN         NaN
2  2021-03-03  114.45  115.90  ...                1.0     NaN         NaN
3  2021-03-02  116.00  116.65  ...                1.0     NaN         NaN
4  2021-03-01  114.40  117.50  ...                1.0     NaN         NaN
5  2021-02-26  115.90  118.40  ...                1.0    3.75    0.033182
6  2021-02-25  115.60  120.50  ...                1.0   -6.30   -0.054394
7  2021-02-24  113.80  115.50  ...                1.0    0.30    0.002636
8  2021-02-23  109.25  114.35  ...                1.0 

Pair Trading: Look at 2 or more stocks with similar traits and use that data to predict movement

In [169]:
#Pair Trading using Cosine Similarity
days = 690
table = []
i_count = 0
j_count = 0
for i in stocks:
  table.append([])
  j_count = 0
  for j in stocks:
      if i == j:
        table[-1].append(1)
        continue
      if i_count > j_count:
        table[-1].append(table[j_count][i_count])
        continue
      data1 = db[i]
      data2 = db[j]
      list_of_tuples = list(zip(data1["timestamp"], data1["log_return"][:days],data2["log_return"][:days]))
      
      tmp = pd.DataFrame(list_of_tuples, columns = ["time" , i , j])
      #tmp.plot(x = "time" , y = [i,j])
      #print(tmp.head(10))
      numerator = tmp[i] * tmp[j]
      numerator = numerator.sum(skipna = True)
      deno = ((tmp[i] ** 2 ).sum(skipna = True))**0.5 * ((tmp[j]**2).sum(skipna = True))**0.5
      #print("NUMERATOR " , deno)
      #print("SIMILARITY INDEX OF " , i , " AND ", j , " IS " , numerator/deno)
      table[-1].append(numerator/deno)
      j_count+=1
i_count = 0
    
print("SIMILARITY TABLE")
table = pd.DataFrame(table,columns = stocks,index=stocks)
print(table)
map_similarity = {}
for i in stocks:
  #print(table.nlargest(3,i))
  same = list(table.nlargest(4,i).index[1:])
  print("TOP 3 stocks like " , i , " are ",same)
  mean = db[same[0]]['log_return'].copy()
  for x in range(len(same)):
    mean+=db[same[x]]['log_return']
  mean/=3
  print("score =  " ,end = "")
  print(mean[5] - db[i]['log_return'][5])
  map_similarity[i] = mean[5] - db[i]['log_return'][5]
print(map_similarity)
  

SIMILARITY TABLE
                ONGC      GAIL    MARUTI  ...  TATASTEEL  HINDALCO       UPL
ONGC        1.000000  0.699631  0.319842  ...   0.581238  0.566895  0.354017
GAIL        0.699631  1.000000  0.350993  ...   0.552170  0.585707  0.415210
MARUTI      0.319842  0.350993  1.000000  ...   0.462458  0.488524  0.459683
HEROMOTOCO  0.323408  0.365177  0.636690  ...   0.321251  0.416104  0.344550
NESTLEIND   0.252852  0.293461  0.437543  ...   0.215924  0.280779  0.349026
ULTRACEMCO  0.449507  0.447082  0.550035  ...   0.515186  0.555634  0.460081
HINDUNILVR  0.276218  0.297187  0.424712  ...   0.242653  0.224925  0.272001
IOC         0.636200  0.561949  0.400358  ...   0.482663  0.449186  0.358474
LT          0.423787  0.460972  0.566122  ...   0.520816  0.599566  0.566311
RELIANCE    0.383129  0.379808  0.377748  ...   0.300404  0.355480  0.433454
BPCL        0.507724  0.501983  0.373870  ...   0.344879  0.408098  0.419716
ADANIPORTS  0.436652  0.424931  0.517684  ...   0.447827  0

In [ ]:
'''
# BOLLINGER BANDS 
days = 365
gap = days//10
mean = []
upar = []
neeche = []
for i in range(days):
  low = i-gap
  if low<0:
    low = 0
  mean.append(data["adjusted_close"][low:i+gap].mean())
  upar.append(mean[-1] + data["adjusted_close"][low:i+5].std())
  neeche.append(mean[-1] - data["adjusted_close"][low:i+5].std())

list_of_tuples = list(zip(data["timestamp"][:days][::-1], data["adjusted_close"][:days][::-1], mean[::-1], upar[::-1] , neeche[::-1]))  
tmp = pd.DataFrame(list_of_tuples,columns=["time","close","mean",'upar','neeche']  )
tmp.plot(x = "time" , y = ["close","mean",'upar','neeche'])

#SIMILARITY / PAIR TRADING

j = 'ONGC'
days = 365
for i in db.keys():
    if i!=j:
      data1 = db[i]
      data2 = db[j]
      list_of_tuples = list(zip(data1["timestamp"][:days][::-1], data1["log_return"][:days][::-1],data2["log_return"][:days][::-1]))
      
      tmp = pd.DataFrame(list_of_tuples, columns = ["time" , i , j])
      tmp.plot(x = "time" , y = [i,j])
  

SyntaxError: ignored